# Encerramento

TODO

## Eu preciso de uma malha de serviços?

TODO

## Eu deveria usar microsserviços?

TODO

## Papel do _gateway_ de API com a malha de serviço?

TODO

![API Gateway and Service Mesh](media/api-gateway-mesh.png)

## Limpeza

Vamos remover tudo que foi instalado no cluster.

In [ ]:
# Apaga todos os recursos do namespace e o namespace
kubectl delete ns financial

# Remove os add-ons (prometheus, graphana, kiali e jaeger)
kubectl delete -f istio-1.8.0/samples/addons

# Remove a aplicação sleep
kubectl delete -f istio-1.8.0/samples/sleep

# Remove todos os compoenntes do istio
istioctl x uninstall --purge -y

# Remove o namespace do istio
kubectl delete ns istio-system

# Remove o rótulo de injeção automática do sidecar do istio
kubectl patch ns default --type=json -p='[{"op": "remove", "path": "/metadata/labels/istio-injection"}]'

# Apaga os recursos criados no curso
kubectl delete -f exemplos/simul-shop/manifests/4

kubectl delete -f exemplos/simul-shop/manifests/8

kubectl delete -f exemplos/simul-shop/manifests/9

kubectl delete -f exemplos/simul-shop/manifests/10